In [ ]:
#!/usr/bin/env python3
#coding=utf-8

from Rosmaster_Lib.Rosmaster import Rosmaster
from ipywidgets import interact
import ipywidgets as widgets
import time

bot = Rosmaster(1, "/dev/USBserial", debug=True)

In [ ]:
# Start to receive data, can only start once, all read data function is based on this method
bot.create_receive_threading()

In [ ]:
# Car Information
version = bot.get_version()
print(version)

car_type = bot.get_car_type_from_machine()
print("car_type:", car_type)

# Read the battery voltage
voltage = bot.get_battery_voltage()
print(voltage)

In [ ]:
# Enable automatic data sending
# If enable=True, the underlying expansion module sends data every 40 milliseconds.  If enable=False, the port is not sent.
# Forever =True for permanent, =False for temporary
enable = True
bot.set_auto_report_state(enable, forever=False)

In [ ]:
# 9-axis
ax, ay, az = bot.get_accelerometer_data()
gx, gy, gz = bot.get_gyroscope_data()
mx, my, mz = bot.get_magnetometer_data()

# print(ax, ay, az, gx, gy, gz, mx, my, mz)

try:
	while True:
		ax, ay, az = bot.get_accelerometer_data()
		gx, gy, gz = bot.get_gyroscope_data()
		mx, my, mz = bot.get_magnetometer_data()
		print(ax, ay, az, gx, gy, gz, mx, my, mz)
		time.sleep(.5)
except KeyboardInterrupt:
	pass

In [ ]:
# Roll Pitch Yaw
roll, pitch, yaw = bot.get_imu_attitude_data()
print("roll:%f, pitch:%f, yaw:%f" % (roll, pitch, yaw))

In [ ]:
# Encoder
# m1, m2, m3, m4 = bot.get_motor_encoder()
# print("encoder:", m1, m2, m3, m4)

try:
    while True:
        m1, m2, m3, m4 = bot.get_motor_encoder()
        print("encoder:", m1, m2, m3, m4)
        time.sleep(.5)
except KeyboardInterrupt:
    pass

In [ ]:
vx, vy, vz = bot.get_motion_data()
print("V:", vx, vy, vz)

In [ ]:
# ID of current steering gear
servo_id = 5

s_id, value = bot.get_uart_servo_value(servo_id)
print("value:", s_id, value)

angle= bot.get_uart_servo_angle(servo_id)
print("angle:", angle)

In [ ]:
#
# set_car_run

Buzzler

In [ ]:
on_time = 1
bot.set_beep(on_time)
time.sleep(1)

Motor Control

In [ ]:
# Control motor movement
def run_motor(M1, M2, M3, M4):
    bot.set_motor(M1, M2, M3, M4)
    return M1, M2, M3, M4


# 创建四个滑块来控制电机 Create four sliders to control the motor
interact(run_motor, \
         M1=widgets.IntSlider(min=-100,max=100,step=1,value=0), \
         M2=widgets.IntSlider(min=-100,max=100,step=1,value=0), \
         M3=widgets.IntSlider(min=-100,max=100,step=1,value=0), \
         M4=widgets.IntSlider(min=-100,max=100,step=1,value=0))

In [ ]:
# 停止运动 stop motion
bot.set_motor(0, 0, 0, 0)

Motion Control

In [ ]:
# 控制电机运动 Control motor movement
def car_motion(V_x, V_y, V_z):
    speed_x= V_x / 1000.0
    speed_y = V_y / 1000.0
    speed_z = V_z / 1000.0
    bot.set_car_motion(speed_x, speed_y, speed_z)
    return speed_x, speed_y, speed_z


# 创建三个滑块来控制小车的速度 Create three sliders to control the speed of the car
interact(car_motion, \
         V_x=widgets.IntSlider(min=-700,max=700,step=1,value=0), \
         V_y=widgets.IntSlider(min=-700,max=700,step=1,value=0), \
         V_z=widgets.IntSlider(min=-3200,max=3200,step=1,value=0))

In [ ]:
# 停止运动 stop motion
bot.set_motor(0, 0, 0, 0)

In [ ]:
# Obtain the linear velocity and angular velocity data of the car
try:
    while True:
        V_x, V_y, V_z = bot.get_motion_data()
        print("speed:", V_x, V_y, V_z)
        bot.clear_auto_report_data()
        time.sleep(.1)
except KeyboardInterrupt:
    pass

In [ ]:
# PID parameter control will affect the set_CAR_motion function to control the speed change of the car.  This parameter is optional by default
kp = 0.8
ki = 0.06
kd = 0.5
bot.set_pid_param(kp, ki, kd, forever=False)

In [ ]:
kp, ki, kd = bot.get_motion_pid()
print("PID:", kp, ki, kd)

Robotic Arm

In [ ]:
servo_id = 4

# 控制串口舵机关节运动 Control the joint movement of serial steering gear
def arm_servo(s_angle):
    bot.set_uart_servo_angle(servo_id, s_angle)

# 创建一个滑块来控制的某个串口舵机 Create a slider to control a serial actuator
if servo_id == 1:
    interact(arm_servo, s_angle=widgets.IntSlider(min=0,max=180,step=1,value=90));
elif servo_id == 2:
    interact(arm_servo, s_angle=widgets.IntSlider(min=0,max=180,step=1,value=90));
elif servo_id == 3:
    interact(arm_servo, s_angle=widgets.IntSlider(min=0,max=100,step=1,value=90));
# elif servo_id == 3:
#     interact(arm_servo, s_angle=widgets.IntSlider(min=0,max=180,step=1,value=90));
elif servo_id == 4:
    interact(arm_servo, s_angle=widgets.IntSlider(min=0,max=180,step=1,value=90));
elif servo_id == 5:
    interact(arm_servo, s_angle=widgets.IntSlider(min=0,max=270,step=1,value=90));
elif servo_id == 6:
    interact(arm_servo, s_angle=widgets.IntSlider(min=30,max=180,step=1,value=90));

In [ ]:
# Read the current serial steering gear Angle, read the Angle value and the set Angle value may be 1-2 degrees deviation
# Read error returns -1
read_servo = bot.get_uart_servo_angle(servo_id)
print("read angle:", servo_id, read_servo)

In [ ]:
# 一次控制的六个串口舵机 Six serial steering gear at one time
def arm_servo(s1, s2, s3, s4, s5, s6):
    bot.set_uart_servo_angle_array([s1, s2, s3, s4, s5, s6])
    return s1, s2, s3, s4, s5, s6

# 创建六个滑块来控制串口舵机的六个关节 Create six sliders to control the six joints of the serial actuator
interact(arm_servo, \
         s1=widgets.IntSlider(min=0,max=180,step=1,value=90), \
         s2=widgets.IntSlider(min=0,max=180,step=1,value=90), \
         s3=widgets.IntSlider(min=0,max=180,step=1,value=90), \
         s4=widgets.IntSlider(min=0,max=180,step=1,value=90), \
         s5=widgets.IntSlider(min=0,max=180,step=1,value=90), \
         s6=widgets.IntSlider(min=30,max=180,step=1,value=180));

In [ ]:
# Read six steering gear angles at one time, the Angle values read and set may be 1-2 degrees deviation
# Read the correct return six steering gear Angle [xx, xx, xx, xx, xx, xx, xx], if one of the steering gear is wrong, which one is -1
read_array = bot.get_uart_servo_angle_array()
print("read array:", read_array)

In [ ]:
# Turn off torque force, the steering gear joint can be turned by hand, but command cannot control rotation
bot.set_uart_servo_torque(False)

In [ ]:
# Turn on torque force, command can control rotation, can not turn steering gear joint by hand
bot.set_uart_servo_torque(True)

In [ ]:
set_servo_id = 2

#输入要写入的伺服ID，1 - 6
#Enter the servo ID to be written, 1 - 6

bot.set_uart_servo_id(set_servo_id)
print("The servo ID has been set to:",set_servo_id)

In [ ]:
# Please delete the object after the program to avoid conflicts caused by Rosmaster library in other programs
del bot

In [ ]:
import cv2
print(cv2.getBuildInformation())

In [6]:
import cv2
cap = cv2.VideoCapture('videotestsrc ! videoconvert ! appsink')
while True:
    ret,frame = cap.read()
    cv2.imshow('',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()
cap.release()

[ WARN:0@67.842] global cap_gstreamer.cpp:1777 open OpenCV | GStreamer warning: Cannot query video position: status=1, value=0, duration=-1


aa

In [7]:
'''
	# How to set video capture properties using V4L2:
	# Full list of Video Capture Properties for OpenCV: https://docs.opencv.org/3.4/d4/d15/group__videoio__flags__base.html
	#Select Pixel Format:
	# video_capture.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'YUYV'))
	# Two common formats, MJPG and H264
	# video_capture.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
	# Default libopencv on the Jetson is not linked against libx264, so H.264 is not available
	# video_capture.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'H264'))
	# Select frame size, FPS:
	video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
	video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
	video_capture.set(cv2.CAP_PROP_FPS, 30)
"""
'''

import cv2

# ASSIGN CAMERA ADDRESS HERE
camera_id = "/dev/video2"

# Full list of Video Capture APIs (video backends): https://docs.opencv.org/3.4/d4/d15/group__videoio__flags__base.html
# For webcams, we use V4L2
# cap = cv2.VideoCapture('/dev/video1')
cap = cv2.VideoCapture(camera_id, cv2.CAP_V4L2)
width = 640
height = 480
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

window_title = "aaa"

In [8]:
if cap.isOpened():
	try:
		window_handle = cv2.namedWindow(window_title, cv2.WINDOW_AUTOSIZE )
		while True:
			ret_val, frame = cap.read()
			# Check to see if the user closed the window
			# Under GTK+ (Jetson Default), WND_PROP_VISIBLE does not work correctly. Under Qt it does
			# GTK - Substitute WND_PROP_AUTOSIZE to detect if window has been closed by user
			if cv2.getWindowProperty(window_title, cv2.WND_PROP_AUTOSIZE) >= 0:
				cv2.imshow(window_title, frame)
			else:
				break
			keyCode = cv2.waitKey(10) & 0xFF
			# Stop the program on the ESC key or 'q'
			if keyCode == 27 or keyCode == ord('q'):
				break
	finally:
		cap.release()
		cv2.destroyAllWindows()
else:
	print("Unable to open camera")

aaaa

In [12]:
import numpy as np
import sys
import cv2 as cv


def main():
	# Open Orbbec depth sensor
	orbbec_cap = cv.VideoCapture(0, cv.CAP_OBSENSOR)
	if orbbec_cap.isOpened() == False:
		sys.exit("Fail to open camera.")

	while True:
		 # Grab data from the camera
		if orbbec_cap.grab():
			# RGB data
			ret_bgr, bgr_image = orbbec_cap.retrieve(None, cv.CAP_OBSENSOR_BGR_IMAGE)
			if ret_bgr:
				cv.imshow("BGR", bgr_image)

			# depth data
			ret_depth, depth_map = orbbec_cap.retrieve(None, cv.CAP_OBSENSOR_DEPTH_MAP)
			if ret_depth:
				color_depth_map = cv.normalize(depth_map, None, 0, 255, cv.NORM_MINMAX, cv.CV_8UC1)
				color_depth_map = cv.applyColorMap(color_depth_map, cv.COLORMAP_JET)
				cv.imshow("DEPTH", color_depth_map)

			# Stop the program on the ESC key or 'q'
			keyCode = cv2.waitKey(10) & 0xFF
			if keyCode == 27 or keyCode == ord('q'):
				break
		else:
			print("Fail to grab data from the camera.")
		keyCode = cv2.waitKey(10) & 0xFF
			# Stop the program on the ESC key or 'q'
		if keyCode == 27 or keyCode == ord('q'):
			break

		if cv.pollKey() >= 0:
			break

	orbbec_cap.release()
	cv.destroyAllWindows()

if __name__ == '__main__':
    main()

[01/19 11:38:32.783058][info][6048][Context.cpp:69] Context created with config: default config!
[01/19 11:38:32.783100][info][6048][Context.cpp:74] Context work_dir=/home/cartoon/rosmaster_ws
[01/19 11:38:32.783139][info][6048][Context.cpp:77] 	- SDK version: 1.9.4
[01/19 11:38:32.783143][info][6048][Context.cpp:78] 	- SDK stage version: main
[01/19 11:38:32.783171][info][6048][Context.cpp:82] get config EnumerateNetDevice:false
[01/19 11:38:32.783414][info][6048][LinuxPal.cpp:38] createObPal: create LinuxPal!
[01/19 11:38:32.945978][warning][6048][OpenNIDeviceInfo.cpp:186] New openni device matched.
[01/19 11:38:32.946631][info][6048][LinuxPal.cpp:112] Create PollingDeviceWatcher!
[01/19 11:38:32.946716][info][6048][DeviceManager.cpp:15] Current found device(s): (1)
[01/19 11:38:32.946730][info][6048][DeviceManager.cpp:24] 	- Name: Astra Pro Plus, PID: 0x060f, SN/ID: , Connection: USB2.0
[01/19 11:38:32.947104][info][6048][Pipeline.cpp:15] Try to create pipeline with default device.


In [1]:
import cv2
print("OpenCV version:", cv2.__version__)

OpenCV version: 4.10.0-dev


In [2]:
import cv2 as cv

orbbec_cap = cv.VideoCapture(0, cv.CAP_OBSENSOR)
if orbbec_cap.isOpened() == False:
	exit("Fail to open camera.")

while True:
	if orbbec_cap.grab():
		# RGB data
		ret_bgr, bgr_image = orbbec_cap.retrieve(None, cv.CAP_OBSENSOR_BGR_IMAGE)
		 # depth data
		ret_depth, depth_map = orbbec_cap.retrieve(None, cv.CAP_OBSENSOR_DEPTH_MAP)

		if ret_depth and ret_bgr:
			color_depth_map = cv.applyColorMap(cv.convertScaleAbs(depth_map, alpha=0.0255), cv.COLORMAP_JET)
			# show the images
			cv.imshow('RBG', bgr_image)
			cv.imshow('Depth', color_depth_map)
		else:
			print("Fail to grab data from the camera.")

		if cv.pollKey():
			break
	keyCode = cv2.waitKey(10) & 0xFF
			# Stop the program on the ESC key or 'q'
	if keyCode == 27 or keyCode == ord('q'):
		break
orbbec_cap.release()

[01/18 19:41:27.283079][info][6036][Context.cpp:69] Context created with config: default config!
[01/18 19:41:27.283124][info][6036][Context.cpp:74] Context work_dir=/home/cartoon/rosmaster_ws
[01/18 19:41:27.283132][info][6036][Context.cpp:77] 	- SDK version: 1.9.4
[01/18 19:41:27.283137][info][6036][Context.cpp:78] 	- SDK stage version: main
[01/18 19:41:27.283148][info][6036][Context.cpp:82] get config EnumerateNetDevice:false
[01/18 19:41:27.283155][info][6036][LinuxPal.cpp:38] createObPal: create LinuxPal!
[01/18 19:41:27.438173][warning][6036][OpenNIDeviceInfo.cpp:186] New openni device matched.
[01/18 19:41:27.438513][info][6036][LinuxPal.cpp:112] Create PollingDeviceWatcher!
[01/18 19:41:27.438680][info][6036][DeviceManager.cpp:15] Current found device(s): (1)
[01/18 19:41:27.438691][info][6036][DeviceManager.cpp:24] 	- Name: Astra Pro Plus, PID: 0x060f, SN/ID: , Connection: USB2.0
[01/18 19:41:27.438704][info][6036][Pipeline.cpp:15] Try to create pipeline with default device.


In [4]:
import numpy as np
import sys
import cv2 as cv


def main():
    # Open Orbbec depth sensor
    orbbec_cap = cv.VideoCapture(0, cv.CAP_OBSENSOR)
    if orbbec_cap.isOpened() == False:
        sys.exit("Fail to open camera.")

    while True:
        # Grab data from the camera
        if orbbec_cap.grab():
            # RGB data
            ret_bgr, bgr_image = orbbec_cap.retrieve(None, cv.CAP_OBSENSOR_BGR_IMAGE)
            if ret_bgr:
                cv.imshow("BGR", bgr_image)

            # depth data
            ret_depth, depth_map = orbbec_cap.retrieve(None, cv.CAP_OBSENSOR_DEPTH_MAP)
            if ret_depth:
                color_depth_map = cv.normalize(depth_map, None, 0, 255, cv.NORM_MINMAX, cv.CV_8UC1)
                color_depth_map = cv.applyColorMap(color_depth_map, cv.COLORMAP_JET)
                cv.imshow("DEPTH", color_depth_map)
        else:
            print("Fail to grab data from the camera.")

        if cv.pollKey() >= 0:
            break

    orbbec_cap.release()


if __name__ == '__main__':
    main()

[01/18 19:42:57.204331][info][6036][Pipeline.cpp:15] Try to create pipeline with default device.
[01/18 19:42:57.204496][info][6036][Pipeline.cpp:44] Pipeline created with device: {name: Astra Pro Plus, sn: ACRC64300Y3}, @0xAAAAED6652D0
[01/18 19:42:57.222670][info][6036][Pipeline.cpp:72] Pipeline destroyed! @0xAAAAED6652D0


[ WARN:0@91.317] global cap.cpp:321 open VIDEOIO(OBSENSOR): raised unknown C++ exception!


[ WARN:0@91.317] global cap.cpp:342 open VIDEOIO(OBSENSOR): backend is generally available but can't be used to capture by index


SystemExit: Fail to open camera.

/home/cartoon/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
